# Store and retrieval


In [1]:
%pip install chromadb
%pip install langchain
%pip install langchain-community
%pip install langchain-chroma
%pip install langchain-huggingface
%pip install langchain-openai


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 1. Load and split the documents

In [2]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# load the document and split it into chunks
loader = TextLoader("./content/max-und-moritz.txt")
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size = 2500, chunk_overlap = 200)
docs = text_splitter.split_documents(documents)

## 2. Store documents into the vector database

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# prepare the embedding model
embedding_function = HuggingFaceEmbeddings(model_name = 'intfloat/multilingual-e5-large')

# load it into Chroma
db = Chroma(embedding_function = embedding_function, persist_directory = './chromadb/')
db.add_documents(docs)

/Users/christianweyer/Sources/gen-ai-customer-workshop-november-2024/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['ad9d978f-6bee-4f1d-8cb2-3b7223443341',
 '41964b37-0ae7-424f-ae73-933661cbb43d',
 'c5051201-5f29-4889-a58b-5d3ca9ed4eaa',
 '67285f68-6606-4c59-b1d5-6c7ec1776aca',
 '1906535a-e571-48e0-85f2-1ad399053857',
 '099fc777-2eb2-4ab6-a16d-f10f574f387e']

## 3. Query the documents

In [ ]:
query = 'Wie heisst der Lehrer?'
#query = 'Who is the sensei?'
#query = 'Was machen die Hühner unmittelbar vor ihrem Tod?'
#query = 'Wie viele Hühner hatte Witwe Bolte?'
#query = 'Was machen Max und Moritz mit dem Schneider und wie heisst er?'

results = db.similarity_search_with_score(query)

for result in results:
    print(f'Score: {result[1]}')
    print(f'Text:\n{result[0].page_content}\n\n')

Score: 0.3036751976994705
Text:
Ihr Gesang wird bang und bänger;

Jedes legt noch schnell ein Ei,

Und dann kommt der Tod herbei. –

Witwe Bolte, in der Kammer,

Hört im Bette diesen Jammer;

Ahnungsvoll tritt sie heraus:

Ach, was war das für ein Graus!

„Fließet aus dem Aug’, ihr Tränen!

All mein Hoffen, all mein Sehnen, Meines Lebens schönster Traum

Hängt an diesem Apfelbaum!!“

Tiefbetrübt und sorgenschwer

Kriegt sie jetzt das Messer her;

Nimmt die Toten von den Strängen,

Daß sie so nicht länger hängen,

Und mit stummem Trauerblick

Kehrt sie in ihr Haus zurück. –

Dieses war der erste Streich,

Doch der zweite folgt sogleich.

Zweiter Streich

Als die gute Witwe Bolte

Sich von ihrem Schmerz erholte,

Dachte sie so hin und her,

Daß es wohl das beste wär’,

Die Verstorb’nen, die hienieden

Schon so frühe abgeschieden,

Ganz im stillen und in Ehren

Gut gebraten zu verzehren. –

– Freilich war die Trauer groß,

Als sie nun so nackt und bloß

Abgerupft am Herde lagen,

Sie, die

## 4. RAG

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

retriever = db.as_retriever()

template = """Beantworte die Frage nur aufgrund der folgenenden Informationen:
{context}

Frage: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model_name = 'gpt-4o'
model = ChatOpenAI(model_name = model_name)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke(query)